In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://fi.wikipedia.org/wiki/Luettelo_Suomen_sairaaloista"
page = requests.get(url)
page

<Response [200]>

In [7]:
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="fi">
 <head>
  <meta charset="utf-8"/>
  <title>
   Luettelo Suomen sairaaloista – Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"fi normal","wgMonthNames":["","tammikuu","helmikuu","maaliskuu","huhtikuu","toukokuu","kesäkuu","heinäkuu","elokuu","syyskuu","lokakuu","marraskuu","joulukuu"],"wgRequestId":"353868b6-564c-48c2-896f-3372c9ad001c","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Luettelo_Suomen_sairaaloista","wgTitle":"Luettelo Suomen sairaaloista","wgCurRevisionId":21030466,"wgRevisionId":21030466,"wgArticleId":809593,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Artikkelit, joissa on laajennusta kaipaavia osia","Luettelot S

In [68]:
def has_numbers(text):
    return any(char.isdigit() for char in text)

result = []

for table in soup.find_all('ul'):
    for element in table.find_all('li'):
        if has_numbers(element.text):
            continue
        #the Pohjola hospital have since changed name into Pihlajalinna    
        if "Pohjola Sairaala" in element.text:
            result.extend(["Pihlajalinna, Helsinki", 
                          "Pihlajalinna, Tampere", 
                          "Pihlajalinna, Oulu", 
                          "Pihlajalinna, Kuopio", 
                          "Pihlajalinna, Turku"])
        #remove named of cities after "Pohjola Sairaala"
        if element.text in ["Helsinki", "Oulu", "Tampere", "Kuopio", "Turku"]:
            continue
        else:
            result.append(element.text)
                
#remove bottom results
stop_index = result.index("Suomen julkisen terveydenhuollon tehohoito on keskitetty suurimpiin sairaaloihin")              
result = result[:stop_index]

Helsingin seudun yliopistollinen keskussairaala, pääsairaalana Meilahden sairaala, Helsinki
Turun yliopistollinen keskussairaala, pääsairaalana Kantasairaala, Turku
Oulun yliopistollinen sairaala, Oulu
Tampereen yliopistollinen sairaala, pääsairaalana Tampereen keskussairaala, Tampere
Kuopion yliopistollinen sairaala, pääsairaalana Puijon sairaala, Kuopio
Kymenlaakson keskussairaala, Kotka
Etelä-Karjalan keskussairaala, Lappeenranta
Päijät-Hämeen keskussairaala, Lahti
Kanta-Hämeen keskussairaala, Hämeenlinna
Satasairaala, Pori
Lapin keskussairaala, Rovaniemi
Vaasan keskussairaala, Vaasa
Seinäjoen keskussairaala, Seinäjoki
Keski-Pohjanmaan keskussairaala, Kokkola
Keski-Suomen Sairaala Nova, Jyväskylä
Mikkelin keskussairaala, Mikkeli
Savonlinnan keskussairaala, Savonlinna
Pohjois-Karjalan keskussairaala, Joensuu
Kainuun keskussairaala, Kajaani
Ahvenanmaan keskussairaala, Maarianhamina
Länsi-Pohjan keskussairaala, Kemi
Grelsby sjukhus, Godby, Finström
Seinäjoen kaupunginsairaala, Seinäjok

In [98]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame (result, columns = ['hospital'])
df.head()

,hospital
0,"Helsingin seudun yliopistollinen keskussairaala, pääsairaalana Meilahden sairaala, Helsinki"
1,"Turun yliopistollinen keskussairaala, pääsairaalana Kantasairaala, Turku"
2,"Oulun yliopistollinen sairaala, Oulu"
3,"Tampereen yliopistollinen sairaala, pääsairaalana Tampereen keskussairaala, Tampere"
4,"Kuopion yliopistollinen sairaala, pääsairaalana Puijon sairaala, Kuopio"


In [100]:
def create_city_column(x):
    chunks = x.split(',')
    if len(chunks) > 1:
        return chunks[-1]
    else:
        return None
        
df["location"] = df["hospital"].apply(create_city_column)

In [101]:
df = df[df.hospital != "Pohjola Sairaala\nHelsinki\nTampere\nOulu\nKuopio\nTurku"]

def add_hospital_network(x):
    if isinstance(x, str):
        if "KYS" in x:
            return "Kuopion yliopistollinen sairaala"
        elif "HYKS" in x:
            return "Helsingin seudun yliopistollinen keskussairaala"
        elif "KHKS" in x:
            return "Kanta-Hämeen keskussairaala"
        elif "TAYS" in x:
            return "Tampereen yliopistollinen sairaala"
        elif "OYS" in x:
            return "Oulun yliopistollinen sairaala"
        elif "TYKS" in x:
            return "Turun yliopistollinen keskussairaala"
        elif "KSKS" in x:
            return "Keski-Suomen keskussairaala"

    
def clean_location_name(x):
    if isinstance(x, str):
        if "(" in x:
            return x.split("(")[0].strip()
        else:
            return x.strip()
    else:
        return "Helsinki"
    
def clean_hospital_name(x):
    if "," in x:
        x = x.split(",")[0].strip()
    if "(" in x:
        x = x.split("(")[0].strip()
    return x.strip()


df["network"] = df["hospital"].apply(add_hospital_network)
df["location"] = df["location"].apply(clean_location_name)
df["hospital"] = df["hospital"].apply(clean_hospital_name)


In [103]:
df.to_csv("hospitals.csv")